# Table of Contents

[Notebook Workflow Step-by-Step for CTX](#notebook-workflow-step-by-step-for-ctx)

[Setup variables and configs](#setup-variables-and-configs)

1. [1. Download images](#1-download-images)
    - [Preview left and right image](#preview-left-and-right-image)
2. [2. First Step of CTX processing lev1eo](#2-first-step-of-ctx-processing-lev1eo)
3. [3. Metadata init](#3-metadata-init)
    - [3.1. Stereo Quality Report](#31-stereo-quality-report)
    - [Downsample images if requested](#downsample-images-if-requested)
4. [4. Bundle adjustment](#4-bundle-adjustment)
5. [5. Stereo first run (steps 1-3 of stereo in ASP)](#5-stereo-first-run-steps-1-3-of-stereo-in-asp)
6. [6. Stereo first run (step 4 of stereo in ASP)](#6-stereo-first-run-step-4-of-stereo-in-asp)
7. [7. Produce low resolution DEM for map projection](#7-produce-low-resolution-dem-for-map-projection)
8. [8. Make GoodPixelMap and Hillshade Previews](#8-make-goodpixelmap-and-hillshade-previews)
    - [Good Pixel Map](#good-pixel-map)
    - [Hillshade of low res DEM](#hillshade-of-low-res-dem)
9. [9. Mapproject ctx against 100m DEM](#9-mapproject-ctx-against-100m-dem)
10. [10. Calculate Better DEM using prior](#10-calculate-better-dem-using-prior)
11. [11. PC alignment (Step 5)](#11-pc-alignment-step-5)
    - [Good Pixel Preview](#good-pixel-preview)
    - [Hillshade of higher res DEM](#hillshade-of-higher-res-dem)
    - [Show pedr data](#show-pedr-data)
12. [12. Start of PC align portion](#12-start-of-pc-align-portion)


# Notebook Workflow Step-by-Step for CTX

Now that we have run the Jupyter Notebook based workflows through the command line interface, we can look at each step that was run and describe what happened in more detail. Note that the function docstrings are also available to describe the parameters of a given step, and what that step does. Below is an export of all the codeblocks in the notebook workflow, additional markdown cells are included in the files but are not important to reproduce here. This workflow replicates the same workflow used by the asp_scripts project.

First define all the parameters for the notebook for papermill. The notebook includes a cell metadata tag for papermill to allow these parameters to be defined at runtime. First we need the left and right image ids, the left image typically has the lower emission angle. ASAP will check the metadata of the images to ensure the correct order is provided. The config1 and config2 parameters are paths to stereo.default files the user has to configure the Ames Stereo Pipeline. The first config file is the only required parameter, config2 gives you to use higher quality parameters for the 2nd pass CTX DEM. The “dem_gsd” and “img_gsd” parameters control the number of pixels per pixel the final DEM and orthorectified images have. These default to 24 and 6 meters per pixel which works for generally any CTX image pair.

Generally, most CTX images are captured at around 5.5 meters per pixel (GSD) so we pick 6 mpp as a reasonable default. By convention, the DEM post spacing should be at least 3X the image GSD. ASAP defaults to 4X the image GSD to be a bit more conservative, resulting in 24 meters per pixel. Output_path is typically left blank to default to the current working directory. The maxdisp parameter controls the maximum expected disparity (distance) between the intermediate CTX DEM and the reference topography. Leaving this as ‘None’ will allow ASAP to estimate the disparity for you. The downsample parameter allows you to downsample the imagery by a factor of the value to reduce processing times, a downsample of 4 will reduce the number of pixels by a factor of 4. The pedr_list variable points to the local copy of a file containing a list of all the paths to all of the MOLA PEDR data. By default this is set to None to use the ODE REST API to grab the necessary PEDR data, which is much faster anyways.


# Setup variables and configs

In [ ]:
from src.asp_wrapper import AmesPipelineWrapper
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Setup the parameters for the notebook

# left, right = 'G03_019456_1646', 'G04_019601_1646'
# left, right = 'K10_057105_1975', 'B08_012627_1975'
# left, right = 'J16_050973_2046', 'N14_068038_2046'
left, right='P20_008788_2067', 'P21_009355_2066'
# left  = 'K14_059010_0916'

# right = 'U15_076642_0916'

# left = 'P02_001981_1823'
# right = 'P03_002258_1817'
config1 = None
config2 = None
dem_gsd  = 24.0
img_gsd  = 6.0
output_path = None
max_disp = None
downsample = None
refdem = None
step_kwargs = {}
# todo: add reference_dem and use to conditional pedr things

In [ ]:
from IPython.display import Image
from pathlib import Path
from src.ctx import CTX
from src.moody import ODE
import math
import os

# default_output_dir = os.path.join(os.getcwd(), 'output')

default_output_dir = "/home/ivan/output/"
# Initialize the ODE and CTX classes
ctx = CTX()
ode = ODE(https=True)
asp = AmesPipelineWrapper()

# Reassign left and right to the correct order
left, right = ctx.get_ctx_order(left, right)

output_path = os.path.join(default_output_dir, f'a_{left}_{right}/') if not output_path else output_path
print("Output directory: ", output_path)

if not os.path.exists(output_path):
    os.makedirs(output_path)

print("Left image PID:  %s\nRight image PID: %s" % (left, right))
print(Path(output_path).absolute())

In [ ]:
print(f'Left image PID: {left}')
data = ode.get_ctx_meta(left)
# print(ode.get_ctx_meta(left))
print(f"Center longitude: {-360 + float(data['Center_longitude'])}")
print(f"Center latitude: {data['Center_latitude']}")

print(f'Right image PID: {right}')
data = ode.get_ctx_meta(right)
# print(ode.get_ctx_meta(left))
print(f"Center longitude: {-360 + float(data['Center_longitude'])}")
print(f"Center latitude: {data['Center_latitude']}")

# 1. Download images


Now we are getting to the heart of the notebook workflow. First use step-one to download our left and right images using the moody tool. At the end of the command you can see we are using standard bash to redirect stdout and stderr to two log files, the first a log just for this step, the second a cumulative log file for the whole job.

`asap ctx step_1 {left} {right} 2>&1 | tee -i -a ./1_download.log ./full_log.log`


In [ ]:
# Download the left and right images using moody

# ctx.step_1()
# ctx.generate_ctx_pair_list(left, right)
left_id = ctx.get_full_ctx_id(left)
right_id = ctx.get_full_ctx_id(right)
# download files
left_path = ode.download_ctx_edr(left_id, output_path)
right_path = ode.download_ctx_edr(right_id, output_path)

left_stem = os.path.join(os.path.dirname(left_path), f'{left_id}')
right_stem = os.path.join(os.path.dirname(right_path), f'{right_id}')

asp.setup_pair_info(output_path, left_stem, right_stem)

# 2. First Step of CTX processing lev1eo

Now we replicate the preprocessing from the asp_scripts project/ames stereo pipeline using ISIS commands. This step will run these steps in the following order:

1) `mroctx2isis`
2) `spiceinit`
3) `spicefit`
4) `ctxcal`
5) `ctxevenodd`

`$ asap ctx step_2 {asap.kwarg_parse(step_kwargs, 'step_2')} 2>&1 | tee -i -a ./2_ctxedr2lev1eo.log ./full_log.log`

Для корректной работы команд необходимо предзагрузить IsisData отсюда: https://astrogeology.usgs.gov/docs/how-to-guides/environment-setup-and-maintenance/isis-data-area/

Т.к. работаем с миссей Mars Reconnaissance Orbiter, нужно ввести `downloadIsisData mro $ISISDATA`

In [ ]:
from src.utils.common import run_parallel, rich_logger


# @rich_logger
# def step_2(ctx_wrapper: CTX, with_web=False):
#     """
#     ISIS3 CTX preprocessing, replaces ctxedr2lev1eo.sh
#
#     :param with_web: if true attempt to use webservices for SPICE kernel data
#     """
files = [output_path + left_id, output_path + right_id]
run_parallel(asp.mroctx2isis, [f'from={i}.IMG to={i}.cub' for i in files], )

run_parallel(asp.spiceinit, [f'from={i}.cub web=yes' for i in files])

run_parallel(asp.spicefit, [f'from={i}.cub' for i in files])
#
run_parallel(asp.ctxcal, [f'from={i}.cub to={i}.lev1.cub' for i in files])

# for cub in cubs:
#     cub.unlink()

"""
**USER ERROR** The CTX image [/mnt/c/Users/Ivan/Documents/PROJECTS/Python/pyameslib/notebooks/output/a_N13_067270_0910_N12_067124_0910/N13_067270_0910_XN_89S005W.lev1.cub] has already been radiometrically calibrated.
"""
# lev1cubs = list(Path(output_path).glob('*.lev1.cub'))

run_parallel(asp.ctxevenodd, [f'from={i}.lev1.cub to={i}.lev1.eo.cub' for i in files])
# for lc in lev1cubs:
#     lc.unlink()


In [ ]:
!ls {output_path}

# 3. Metadata init

Now we create a number of metadata files used by the asp_scripts project to simplify future command calls. We also copy our preprocessed CTX cub files into a new working directory where all the stereo products will be computed. This new directory name uses both image IDs joined by an underscore ‘{left_id}_{right_id}’, for example: “B03_010644_1889_XN_08N001W_P02_001902_1889_XI_08N001W”.

`$ asap ctx step_3`

## 3.1. Stereo Quality Report

In [ ]:
# Generate stereo quality

qual_report = asp.get_stereo_quality_report(f'{output_path + left_id}.lev1.eo.cub', f'{output_path + right_id}.lev1.eo.cub')
print(qual_report)

### Downsample images if requested

In [ ]:
if downsample:
    true_img_gsd_left = asp.get_image_gsd(f'{left}_{right}/{left}.lev1eo.cub')
    true_img_gsd_right = asp.get_image_gsd(f'{left}_{right}/{right}.lev1eo.cub')
    # take conservative approach, pick worst image GSD
    res_gsd = max(true_img_gsd_left, true_img_gsd_right)
    # this is because rescale in ISIS does not update GSD in metadata
    asp.rescale_and_overwrite(factor=downsample)
    img_gsd = math.ceil(res_gsd)*downsample
    dem_gsd = 4*img_gsd
    print('new img gsd', img_gsd)
    print('new dem gsd', dem_gsd)

In [ ]:
!gdal_translate -of GTiff -co COMPRESS=LZW {left_stem}.lev1.eo.cub {left_stem}.tif
!gdal_translate -of GTiff -co COMPRESS=LZW {right_stem}.lev1.eo.cub {right_stem}.tif

# 4. Bundle adjustment

We will use the parallel_bundle_adjust command from Ames Stereo Pipeline to refine the spacecraft position and orientation. The user can later re-run this step with more advanced options or GCPs if so desired.

`!asap ctx step_4 {asap.kwarg_parse(step_kwargs, 'step_4')} 2>&1 | tee -i -a ./2_bundle_adjust.log ./full_log.log`

In [ ]:
"""
Bundle Adjust CTX

Run bundle adjustment on the CTX map projected data

:param vargs: variable length additional positional arguments to pass to bundle adjust
:param bundle_adjust_prefix: prefix for bundle adjust output
:param postfix: postfix for cub files to use
:param camera_postfix: postfix for cameras
"""
asp.bundle_adjust(
    bundle_adjust_prefix='adjust/ba',
    postfix='.lev1.eo.cub',
    camera_postfix='.lev1.eo.json'
)

## Preview

In [ ]:
!gdal_translate -of GTiff {left_stem}.lev1.eo.cub {left_stem}.tif -co COMPRESS=LZW
!gdal_translate -of GTiff {right_stem}.lev1.eo.cub {right_stem}.tif -co COMPRESS=LZW

In [ ]:
# Show matplotlib previews
import matplotlib.pyplot as plt
import rasterio
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, dpi=400)

with rasterio.open(f'{left_stem}.tif', 'r') as src:
    data = src.read(1)
    img1 = ax1.imshow(src.read(1), cmap='gray', vmin=np.min(data), vmax=np.max(data))
    ax1.set_xticks([])  # Remove x-axis ticks
    ax1.set_yticks([])
    ax1.set_title(f'{left}.IMG')

with rasterio.open(f'{right_stem}.tif', 'r') as src:
    data = src.read(1)
    img2 = ax2.imshow(data, cmap='gray', vmin=np.min(data), vmax=np.max(data))
    ax2.set_xticks([])  # Remove x-axis ticks
    ax2.set_yticks([])
    ax2.set_title(f'{right}.IMG')

fig.subplots_adjust(wspace=0.1)
plt.show()

# 5. Stereo first run (steps 1-3 of stereo in ASP)

Now we can start making our first dem, we pass in the stereo config file to parallel_stereo. We split this into two parts (step 5 & 6) as we may want to run each part with slightly different parameters or give us a chance to inspect the outputs before the final step which can be long running. In the future Step 5 & & maybe reconfigured into the 4 sub-steps for further improvement to the workflow.

`!asap ctx step_5 {config1} {asap.kwarg_parse(step_kwargs, 'step_5')} 2>&1 | tee -i -a ./3_lev1eo2dem.log ./full_log.log`

In [ ]:
"""
Parallel Stereo Part 1

Run first part of parallel_stereo asp_ctx_lev1eo2dem.sh

:param postfix: postfix for cub files to use
:param camera_postfix: postfix for cameras  # TODO: use .adjusted_state.json?
"""
# defaults_ps1 = {
#     '--processes': _processes,
#     '--threads-singleprocess': _threads_singleprocess,
#     '--threads-multiprocess': _threads_multiprocess,
#     '--stop-point': 5,
#     '--bundle-adjust-prefix': 'adjust/ba'
# }

# asp.stereo_asap(cub_postfix='.lev1.eo.cub', cam_postfix='.lev1.eo.json', stop_point=5)
print(asp.workdir)
for step in range(0, 5):
    print(f"Running step {step} of stereo")
    # run the step
    asp.stereo_asap(
        cub_postfix='.lev1.eo.cub',
        cam_postfix='.lev1.eo.json',
        entry_point=step,
        stop_point=step + 1,
        run='results_raw',
        output_file_prefix="${run}/out",
        stereo_algorithm='asp_mgm',
        corr_memory_limit_mb=16384,
    )
# asp.stereo_asap(
#     cub_postfix='.lev1.eo.cub',
#     cam_postfix='.lev1.eo.json',
#     entry_point=0,
#     stop_point=5,
#     # alignment_method='local_epipolar',
#     stereo_algorithm='asp_mgm',
#     # job_size_h=512,
#     # job_size_w=512,
#     # sgm_collar_size=128,
#     corr_memory_limit_mb=16384
# )

# 6. Stereo first run (step 4 of stereo in ASP)

Run step 4, see step 5 above for more information.

`!asap ctx step_6 {config1} {asap.kwarg_parse(step_kwargs, 'step_6')}  2>&1 | tee -i -a ./3_lev1eo2dem.log ./full_log.log`

In [ ]:
# Parallel Stereo Part 2
#
# Run second part of parallel_stereo, asp_ctx_lev1eo2dem.sh stereo is completed after this step
#
# :param postfix: postfix for cub files to use
# :param camera_postfix: postfix for cameras  # TODO: use .adjusted_state.json?

# defaults_ps2 = {
    #     '--processes': _threads_singleprocess,  # use more cores for triangulation!
    #     '--threads-singleprocess': _threads_singleprocess,
    #     '--threads-multiprocess': _threads_multiprocess,
    #     '--entry-point': 5,
    #     '--bundle-adjust-prefix': 'adjust/ba'
    # }

# return self.cs.stereo_asap(stereo_conf, postfix=postfix, camera_postfix=camera_postfix,
#                            posargs=posargs, **{**self.cs.defaults_ps2, **kwargs})
asp.stereo_asap(cub_postfix='.lev1.eo.cub', cam_postfix='.lev1.eo.json', entry_point=5, stop_point=6, run='results_raw')

# 7. Produce low resolution DEM for map projection

We have made a point cloud, but it is preliminary so we will use it to make a 100 mpp DEM to map-project the CTX images to, to produce a better 2nd pass DEM.


`! asap ctx step_7 --mpp 100 --just_ortho False --dem_hole_fill_len 50 {asap.kwarg_parse(step_kwargs, 'step_7')} 2>&1 | tee -i -a ./4_make_100m_dem.log ./full_log.log`

In [ ]:
"""
Produce preview DEMs/Orthos

Produce dem from point cloud, by default 24mpp for ctx for max-disparity estimation

:param run: folder for results
:param just_ortho: set to True if you only want the ortho image, else make dem and error image
:param mpp: resolution in meters per pixel
:param postfix: postfix for cub files to use
"""
# return self.cs.point_to_dem(mpp, 'PC.tif',
#                             just_ortho=just_ortho,
#                             postfix=postfix,
#                             run=run,
#                             kind='ba',
#                             use_proj=self.proj,
#                             **kwargs)
asp.point_to_dem(100, 'PC.tif', just_ortho=False, run='results_raw', dem_hole_fill_len = 50)

## Make GoodPixelMap and Hillshade Previews
We make image previews of the DEM using the next few steps to check for issues with our first pass DEM. First we will render out the good pixel map image and then the hillshade of the DEM to look for issues with the topography.

`!asap ctx step-8`

In [ ]:
"""
hillshade First step in asp_ctx_step2_map2dem script

:param output_folder:
:param run:
"""
_left_img, _right_img = asp._get_pair()
dem_workdir = asp.stereo_pair.workdir + 'dem/'
dem: Path = next(Path(dem_workdir).glob('*DEM.tif'))
asp.hillshade(dem.absolute(), f'{dem_workdir}/{dem.stem}-hillshade.tif')

### Good Pixel Map

Use some python to specify a new file name for the png version

In [ ]:
# both = f'{left}_{right}'
img = f'{asp.stereo_pair.workdir}/results/out-GoodPixelMap.tif'
out = img.replace('.tif', '.png')

Use gdal_translate to produce a png version of the hillshade image.

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

### Hillshade of low res DEM

Now again for the hillshade

In [ ]:
# both = f'{left}_{right}'
img = f'{dem_workdir}/{dem.stem}-hillshade.tif'
out = img.replace('.tif', '.png')

Convert to a png file again.

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

Display the image in the notebook.

In [ ]:
import matplotlib.pyplot as plt
import rasterio

# Пути к вашим GeoTIFF-файлам
# img1_path = 'путь_к_первому_изображению.tif'
# img2_path = 'путь_ко_второму_изображению.tif'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

with rasterio.open(asp.stereo_pair.workdir + 'results_raw/out-GoodPixelMap.tif') as src1:
    ax1.imshow(src1.read(1))
    ax1.set_title('Good Pixel Map')
    ax1.axis('off')

with rasterio.open(dem_workdir + f'/{dem.stem}-hillshade.tif') as src2:
    ax2.imshow(src2.read(1), cmap='gray')
    ax2.set_title('Hillshade')
    ax2.axis('off')

plt.tight_layout()
plt.show()

# 8. Mapproject ctx against 100m DEM

We now map-project our ctx images against our low resolution DEM to reduce image distortion for our 2nd pass DEM.

`!asap ctx step_9 --mpp {img_gsd} {asap.kwarg_parse(step_kwargs, 'step_9')} 2>&1 | tee -i -a ./5_mapproject_to_100m_dem.log ./full_log.log`

In [ ]:
""""
Mapproject the left and right ctx images against the reference DEM

:param run: name of run
:param refdem: reference dem to map project using
:param mpp: target GSD
:param postfix: postfix for cub files to use
:param camera_postfix: postfix for cameras to use
"""
asp.mapproject_both(ref_dem=dem.absolute(), mpp=img_gsd, bundle_adjust_prefix='adjust/ba')

# 9. Calculate Better DEM using prior

Same as step 5, just using the new map projected images this time.

`!asap ctx step_10 {config2} {asap.kwarg_parse(step_kwargs, 'step_10')} 2>&1 | tee -i -a ./6_next_level_dem.log ./full_log.log`

In [ ]:
asp.stereo_asap(
    run='results_map_ba',
    cub_postfix='.ba.map.tif',
    cam_postfix='.lev1.eo.json',
    ref_dem=dem.absolute(),
    entry_point=0,
    stop_point=5,
    stereo_algorithm='asp_mgm',
    subpixel_mode=3,
    corr_memory_limit_mb=16384,
)

Same as step 6, just using the new map projected images this time.

`!asap ctx step_11 {config2} {asap.kwarg_parse(step_kwargs, 'step_11')} 2>&1 | tee -i -a ./6_next_level_dem.log ./full_log.log`

In [ ]:
asp.stereo_asap(
    run='results_map_ba',
    cub_postfix='.ba.map.tif',
    cam_postfix='.lev1.eo.json',
    ref_dem=dem.absolute(),
    entry_point=5,
    stop_point=6,
    stereo_algorithm='asp_mgm',
    subpixel_mode=3,
    corr_memory_limit_mb=16384,
)

We have made our second point cloud, so we should export some visuals as before. The parameter ‘–folder’ just specifies that we are saving things into a different directory this time around.

`!asap ctx step_7 --mpp {dem_gsd} --run results_map_ba {asap.kwarg_parse(step_kwargs, 'step_7_2')}`



In [ ]:
asp.point_to_dem(24, 'PC.tif', run='results_map_ba')

`!asap ctx step_8 --run results_map_ba`

In [ ]:
_left_img, _right_img = asp._get_pair()
dem_workdir = asp.stereo_pair.workdir + 'dem/'
dem: Path = next(Path(dem_workdir).glob('*24_0-DEM.tif'))
asp.hillshade(dem.absolute(), f'{dem_workdir}/{dem.stem}-hillshade.tif')

## Make Final GoodPixelMap and Hillshade Previews

### Good Pixel Map


Nothing too surprising here, just export PNG versions of the images we care about to see the DEM at this stage of the processing.

In [ ]:
both = f'{left}_{right}'
img = f'./{both}/results_map_ba/{both}_ba-GoodPixelMap.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out, width=800)

### Hillshade of higher res DEM

In [ ]:
both = f'{left}_{right}'
img = './' + str(next(Path('../src/pyameslib/').glob(f'./{both}/results_map_ba/dem/{both}_ba_*-DEM-hillshade.tif')))
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out, width=600)

# 11. PC alignment (Step 5)
The final important step in the make_dem workflow is to get the MOLA PEDR data for the region we care about. Again, our data is not completely done until it has been aligned to the MOLA topography. If we had GCPs in the bundle adjust stage this would not be as big of an issue, but since it is relatively easy to align to MOLA we don’t need to go through the process of producing GCPs.

There are two possibilities, either refdem is none (in which case get pedr data using moody) or we are given a dem
currently this will always run even if refdem is provided, but below pc_align call will use refdem if it's not none

`!asap ctx step_12 {refdem} 2>&1 | tee -i -a ./7_pedr_for_pc_align.log ./full_log.log`

In [ ]:
asp.get_pedr_4_pcalign_w_moody(
    cub_path=asp.stereo_pair.left + '.lev1.eo.cub',
    proj=ctx.proj,
    output_prefix="pedr4align"
)

## Show pedr data

One additional bit here, for the MOLA data, show the PEDR2TAB template if created and the amount of PEDR data we have to align to. If the final line is less than a few hundred we could be in a bad situation.

In [ ]:
# !cat {asp.stereo_pair.workdir}PEDR2TAB.PRM
import csv

with open(asp.stereo_pair.workdir + 'pedr4align.csv', 'r') as f:
    reader = csv.reader(f)
    pedr_data = list(reader)
    print(f"Number of PEDR data points: {len(pedr_data)}")

In [ ]:
# !cat ./{left}_{right}/{left}_{right}_pedr4align.csv | wc -l

Now that we have finished the first half of the workflow we can inspect the output products for issues before moving forwards. If there are issues noted in the log or after a particular step, that step can be re-run with different parameters until a good solution is found.

At this point, we have a completed DEM! However, it’s absolute position in space maybe off from the correct position. Therefore, we must now perform a point cloud alignment to align our DEM with reference topography, in this case MOLA PEDR data to correct the position of the CTX DEM. In older versions of ASAP, this point is the dividing line between the make_dem and align_dem pipelines.

The “maxdisp” parameter in particular deserves attention. It is the number passed to pc_align's –max-displacement parameter in the Ames Stereo Pipeline. Basically, it is the value of the distance you expect to move the CTX DEM to become aligned to your reference DEM (in this case, the PEDR data). It is generally worth estimating this number using a GIS to sample points in both the DEM and reference file, and seeing how far away they are from each other. But, CTX can be well-behaved with ASP, so we pick a default of 500 meters which can be large enough for many situations.

# 12. Start of PC align portion

### Align the DEM to MOLA

This is the most important step in the 2nd half of the workflow as all the remaining steps are just producing final science products and visuals for the logs. This step runs pc_align using the provided max displacement (aka disparity). If the logs indicate a larger displacement was observed than the user provided value it will need to be re-run using larger values or with other advanced parameters. If users see issues it is generally easyier to re-run the pipeline at this step repeatedly in the command line or inside the Jupyter notebook.

`!asap ctx step_13 --maxd {max_disp} --refdem {refdem} {asap.kwarg_parse(step_kwargs, 'step_13')} 2>&1 | tee -i -a ./8_pc_align.log ./full_log.log`


In [ ]:

asp.point_cloud_align(
    datum="D_MARS",
    max_disparity=1000,
    src_dem =asp.workdir +"results_map_ba/out-PC.tif",
    ref_dem=asp.workdir + "pedr4align.csv",
    kind='map_ba_align',
)

### Make the final CTX DEM

After the previous step everything after is simple and easy as we now have a final aligned point cloud from which DEMs and ortho images can be made. That is all the rest of the steps do, they generate final DEMs with the geoid adjustment to produce science ready DEMs and ortho images for mapping.

`!asap ctx step_14 --mpp {demgsd}  2>&1 | tee -i -a ./9_dems_orthos.log ./full_log.log`

In [ ]:
asp.point_to_dem(
    mpp=img_gsd,
    pc_suffix = 'trans_reference.tif',
    just_ortho=False,
    run='pc_align',
    kind='map_ba_aligned',
    use_proj=ctx.proj,
)

### Adjust final CTX DEM to Geoid (Areoid)

`!asap ctx step_15 2>&1 | tee -i -a ./10_geoid_adjustment.log  ./full_log.log`


In [ ]:
asp.geoid_adjust(
    src_dem_sfx = "6_0-DEM.tif",
    run = asp.workdir + "dem"
)

### Make the final CTX Hillshade and Orthos

`!asap ctx step_8 --folder results_map_ba --output_folder dem_align 2>&1 | tee -i -a ./11_hillshade.log ./full_log.log`

In [ ]:
_left_img, _right_img = asp._get_pair()
dem_workdir = asp.stereo_pair.workdir + 'dem/'
dem: Path = next(Path(dem_workdir).glob('*6_0-DEM-geoid-adj.tif'))
asp.hillshade(dem.absolute(), f'{dem_workdir}/{dem.stem}-hillshade.tif')

In [ ]:
img = f'{dem_workdir}/{dem.stem}-hillshade.tif'
out = img.replace('.tif', '.png')

In [ ]:
!gdal_translate -of PNG -co worldfile=yes {img} {out}

In [ ]:
Image(filename=out, width=800)

`!asap ctx step_14 --mpp {img_gsd} --just_ortho True  2>&1 | tee -i -a ./12_img_full_ortho.log ./full_log.log`


In [ ]:
asp.point_to_dem(
    mpp=img_gsd,
    pc_suffix = 'out-PC.tif',
    just_ortho=True,
    run='results_map_ba',
    kind='map_ba_aligned_ortho',
    use_proj=ctx.proj,
)